<img src="https://raw.githubusercontent.com/kubinka0505/matchering-cli/master/Documents/Pictures/Matchering_CLI.svg" width=500>
<br>&emsp;

---

<br>

<a href="https://github.com/kubinka0505/matchering-cli"><img src="https://img.shields.io/badge/github-open-dimgray?logoColor=FFF&logo=github&style=for-the-badge"></a>

# <font color=lime>**1.**</font> Setup ⚙️

In [ ]:
#@title ## Re/install requirements 📥
import os
from shutil import rmtree

__title__ = "matchering-cli"
__title_short__ = __title__[::3].upper()

if "__BaseDir" in locals():
	os.chdir(__BaseDir)

if os.path.exists(__title__):
	rmtree(__title__)

Packages = ["ffmpeg"]

#-=-=-=-#

!apt-get update
for Package in Packages:
	!apt-get install {Package} | grep "{Package}"

print("\n{0}\n".format("━" * 64))
!git clone https://github.com/kubinka0505/matchering-cli
os.chdir(__title__)

print("\n{0}\n".format("━" * 64))
!pip install rich -r requirements.txt | grep -v "already"
os.chdir("..")

#-=-=-=-#

__BaseDir = "."

In [ ]:
#@title ## Setup code ⚙️
import requests
from time import time
from pathlib import Path
from zipfile import ZipFile
from rich import print as rprint

try:
	from mutagen import File as mFile
except ImportError:
	raise SystemExit("Requirements were not installed.")

try:
	from google.colab import files
	Colab = 1
except ImportError:
	Colab = 0

#-=-=-=-#

__BaseDir = os.path.abspath(__BaseDir)
os.chdir(__BaseDir)

Uploads_Directory = os.path.abspath("Uploads")
Targets_Directory = os.path.abspath("Uploads/Targets")
References_Directory = os.path.abspath("Uploads/References")
Results_Directory = os.path.abspath("Uploads/Results")

for Directory in Uploads_Directory, Targets_Directory, References_Directory:
	os.makedirs(Directory, exist_ok = 1)

#-=-=-=-#

def File_Size(Bytes: float) -> str:
	"""Returns human-readable file size."""
	for Unit in ("", "K", "M", "G", "T"):
		if Bytes < 1024:
			break
		Bytes /= 1024

	Bytes = round(Bytes, 2) if Bytes else 0
	return str(Bytes)[::-1].zfill(4)[::-1] + " " + Unit + "B"

# <font color=gold>**2.**</font> Audio files 🔊

## Setup <font color=lime>**TARGET**</font> 🎶
> *Audio files you want to master.*

In [ ]:
#@title ### Remove <font color=lime>**TARGET**</font> tracks 🗑️

Files = [str(File) for File in Path(Targets_Directory).glob("*.*")]
__LF = len(Files)

for File in Files:
	os.remove(File)

rprint("Removed [bold #00CC00]{0}[/] file{1}.".format(
	__LF,
	"" if __LF == 1 else "s"
))

In [ ]:
#@title ### Fetch <font color=lime>**TARGET**</font> 📥
#@markdown ℹ️ Leave empty for manual upload.<br>
#@markdown ℹ️ Multiple URLs separated by `,` supported.
Direct_URLs = "" #@param {type: "string"}
Direct_URLs = Direct_URLs.split(",")
Direct_URLs = [URL.strip() for URL in set(Direct_URLs)]
Direct_URLs = [URL for URL in Direct_URLs if URL]
__LDU = len(Direct_URLs)

if not "os" in locals():
	raise SystemExit("Requirements were not installed.")

os.chdir(Targets_Directory)

if Direct_URLs:
	for Direct_URL in Direct_URLs:
		# Normalize URL
		for Delimiter in "#", "?", "&":
			Direct_URL = Direct_URL.split(Delimiter)[0]

		Output_File = Direct_URL.split("/")[-1]
		Output_File = Output_File.replace(" ", "_")

		with requests.head(Direct_URL, allow_redirects = True) as Site:
			if Site.ok:
				if "audio" in Site.headers["content-type"].lower():
					rprint(f'[bold #44AAFF]Downloading [#FFCC00]"{Output_File}"[/][bold]...[/]')
					!curl -# "{Site.url}" --output "{Output_File}"

					if all((__LDU > 0, Direct_URLs.index(Direct_URL) + 1 != __LDU)):
						print()
				else:
					rprint("[bold #FF0000]Not an audio file[/]")
			else:
				rprint('[bold #FF0000]URL returned status code {1} ("{2}")[/]'.format(
					Site.status_code, Site.reason,
				))
elif Colab:
	rprint("Upload [bold][#00CC00]TARGET [#44AAFF]AUDIO[/][/][/] files.")
	Files = list(files.upload().keys())
	for File in Files:
		if not "audio" in mFile(File).mime[0]:
			Files.remove(File)
			os.remove(File)

os.chdir(__BaseDir)

---

## Setup <font color=gold>**REFERENCE**</font> 🔎
> *Reference track.*

In [ ]:
#@title ### Remove <font color=gold>**REFERENCE**</font> track 🗑️

Files = [str(File) for File in Path(References_Directory).glob("*.*")]
__LF = len(Files)

for File in Files:
	os.remove(File)

rprint("Removed [bold #00CC00]{0}[/] file{1}.".format(
	__LF,
	"" if __LF == 1 else "s"
))

In [ ]:
#@title ### Fetch <font color=gold>**REFERENCE**</font> 📥
#@markdown ℹ️ Leave empty for manual upload.
Direct_URL = "" #@param {type: "string"}

if not "os" in locals():
	raise SystemExit("Requirements were not installed.")

os.chdir(References_Directory)

if Direct_URL:
	# Normalize URL
	for Delimiter in "#", "?", "&":
		Direct_URL = Direct_URL.split(Delimiter)[0]

	Output_File = Direct_URL.split("/")[-1]
	Output_File = Output_File.replace(" ", "_")

	with requests.head(Direct_URL, allow_redirects = True) as Site:
		if Site.ok:
			if "audio" in Site.headers["content-type"].lower():
				rprint(f'[bold #44AAFF]Downloading [#FFCC00]"{Output_File}"[/][bold]...[/]')
				!curl -# "{Site.url}" --output "{Output_File}"
			else:
				rprint("[bold #FF0000]Not an audio file[/]")
		else:
			rprint('[bold #FF0000]URL returned status code {1} ("{2}")[/]'.format(
				Site.status_code, Site.reason,
			))
elif Colab:
	rprint("Upload [bold][#FFCC00]REFERENCE [#44AAFF]AUDIO[/][/][/] file.")
	Reference = list(files.upload())
	for File in Reference:
		if not "audio" in mFile(File).mime[0]:
			rprint("[bold #FF0000]Reference file is not audio file, removing.[/]")
			Files.remove(File)
			os.remove(File)
		else:
			Reference = os.path.join(References_Directory, Reference[0])

os.chdir(__BaseDir)

# <font color=gold>**3.**</font> Program</font>

In [ ]:
#@title ## Config ⚙️
#@markdown > *`FLAC` output format is forced.*
#@markdown ---

Bit_Depth = 24 #@param {type: "slider", min: 16, max: 32, step: 8}

Sample_Rate = "Auto" #@param ["8 000 Hz", "16 000 Hz", "32 000 Hz", "44 100 Hz", "48 000 Hz", "88 200 Hz", "96 000 Hz", "192 000 Hz", "Auto"]
Sample_Rate = Sample_Rate.replace(" ", " Hz").replace(" Hz", "").lower()

FFT_Window_Size = "4096" #@param ["256", "512", "1024", "2048", "4096", "8192", "16384", "32768", "65536", "131072", "262144"]

Logging_Level = "Info" #@param ["Info", "Warnings", "Debug"]
Logging_Level = {"info": 0, "warnings": 1, "debug": 2}[Logging_Level.lower()]

#@markdown ---

Limiter = True #@param {type: "boolean"}
Normalize = True #@param {type: "boolean"}

#-=-=-=-#

os.environ["BIT_DEPTH"] = str(Bit_Depth)
os.environ["RATE"] = Sample_Rate
os.environ["FFT"] = FFT_Window_Size
os.environ["LOGLEVEL"] = str(Logging_Level)
os.environ["LIMITER"] = "" if Limiter else "-nl"
os.environ["NORMALIZE"] = "" if Normalize else "-nn"

In [ ]:
#@title ## Run ▶️
Only_Print_Command = False #@param {type: "boolean"}
os.chdir(os.path.join(__BaseDir, __title__))

Target_Results_Directory = os.path.join(
	Results_Directory,
	"_".join((
		__title_short__, str(time()).split(".")[-1].zfill(7)[::-1]
	))
)

#-=-=-=-#

if not os.path.exists(Reference):
	raise SystemExit("Reference file was not found!")

os.environ["PROG"] = ("echo " if Only_Print_Command else "") + os.sys.executable
os.environ["RESULTS"] = Target_Results_Directory
os.environ["REFERENCE"] = Reference
os.environ["PYFILE"] = [File for File in os.listdir() \
	if ".py" in os.path.splitext(File)[-1].lower()
][-1]

for File in Path(Targets_Directory).glob("*.*"):
	os.environ["INPUT"] = str(File.resolve())
	!$PROG "$PYFILE" -f flac -nfd -v $LOGLEVEL \
		-i "$INPUT" -r "$REFERENCE" -o "$RESULTS" \
		-sr $RATE -fft $FFT \
		$LIMITER $NORMALIZE

---

# Utils ⚙️

In [ ]:
#@title Download files as `ZIP` archive 📥
Archive_Path = "_".join((
	__title__, str(time()).split(".")[0].zfill(8)
)) + ".zip"

with ZipFile(Archive_Path, "w", 14, 1, 9) as Archive:
	for File in Path(Results_Directory).rglob("*"):
		File = str(File.resolve())
		File_Relative = File.split(Results_Directory)[-1][1:]

		if os.path.isfile(File):
			rprint('\t[bold #44AAFF]Writing [#FFCC00]"{0}{}"[/]...[/]'.format(
				os.path.join(
					os.path.basename(Archive_Path),
					File_Relative
				)
			))
			Archive.write(File, File_Relative)
print()

if Colab:
	files.download(Archive_Path)
else:
	rprint("[bold #00CC00]Done![/]")